In [ ]:
import pandas as pd
netflix=pd.read_csv('../input/updated-netflix-stock-price-all-time/netflix.csv')
netflix.head()

In [ ]:
netflix=netflix[['Date','Close']]
netflix.head()

In [ ]:
netflix.info()

In [ ]:
!pip install pycaret-ts-alpha

In [ ]:
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment
from sktime.utils.plotting import plot_series

In [ ]:
netflix.index = pd.DatetimeIndex(netflix["Date"])
netflix.drop(["Date"],axis=1,inplace=True)
netflix.head()

In [ ]:
netflix = netflix.asfreq('d')
netflix.index

In [ ]:
netflix.head()

In [ ]:
netflix=netflix.fillna(method='ffill')
netflix.head()

In [ ]:
data=netflix.Close
data

In [ ]:
import matplotlib.pyplot as plt
_ =plot_series(data)
plt.xticks(rotation=90)
plt.grid()

In [ ]:
exp = TimeSeriesExperiment()
exp.setup(data=data,session_id=42,fh=365)

In [ ]:
## Create a model manually
model = exp.create_model("arima")
model

In [ ]:
y_train = exp.get_config("y_train")
y_train[:5]

In [ ]:
y_predict = exp.predict_model(model)
plot_series(data, y_train, y_predict, labels=['All', 'Train', 'Test Predictions'])
plt.grid()

In [ ]:
tuned_model = exp.tune_model(model,optimize = 'RMSE')
tuned_model

In [ ]:
exp.predict_model(tuned_model)

In [ ]:
y_predict = exp.predict_model(tuned_model)
plot_series(data, y_train, y_predict, labels=['All', 'Train', 'Test Predictions'])
plt.grid()

In [ ]:
# Finalize Model
final_model = exp.finalize_model(tuned_model)
# Future Predictions
y_pred_future = exp.predict_model(final_model, verbose=False)
_=plot_series(data, y_pred_future, labels=['All', 'Future Pred (Unknown)'])
plt.grid()

In [ ]:
# Test Set Predictions
y_pred = exp.predict_model(tuned_model)

# Finalize Model
final_model = exp.finalize_model(tuned_model)

# Future Predictions
y_pred_future = exp.predict_model(final_model, verbose=False)

# Plot Results
y_train = exp.get_config("y_train")
_ = plot_series(data, y_train, y_pred, y_pred_future, labels=['All', 'Train', 'Test Predictions', 'Future Pred (Unknown)'])
plt.grid()

# 